In [121]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from utils import split_sequence, get_apple_close_price, plot_series

In [122]:
apple_close_price = get_apple_close_price()

In [123]:
from sklearn.model_selection import train_test_split
from scipy.stats import boxcox
from scipy.special import inv_boxcox

In [124]:
train, test = train_test_split(apple_close_price, 
                               test_size=0.05, 
                               shuffle=False)

In [125]:
from sklearn.preprocessing import MinMaxScaler

In [126]:
scaler = MinMaxScaler()

In [127]:
scaled_train = scaler.fit_transform(train.values.reshape(-1, 1))
scaled_test = scaler.transform(test.values.reshape(-1, 1))

In [128]:
# Core layers
from keras.layers \
    import Activation, Dropout, Flatten, Dense, Input, LeakyReLU, Reshape

# Recurrent layers
from keras.layers import LSTM

# Convolutional layers
from keras.layers import Conv1D

# Normalization layers
from keras.layers import BatchNormalization

# Merge layers
from keras.layers import concatenate

# Layer wrappers
from keras.layers import Bidirectional, TimeDistributed

# Keras models
from keras.models import Model, Sequential

# Keras optimizers
from keras.optimizers import Adam, RMSprop

import keras.backend as K

import warnings
warnings.simplefilter('ignore')

In [129]:
look_back = 3
n_features = 1

In [130]:
X_train, y_train = split_sequence(scaled_train, look_back)
X_test, y_test = split_sequence(scaled_test, look_back)

In [131]:
def build_generator(look_back, n_features):
    model = Sequential()

    model.add(LSTM(50, return_sequences=True, input_shape=(look_back, n_features)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(LSTM(100))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Dense(n_features))

    print('Generator summary:')
    model.summary()
    
    return model
    
#     seq = Input(shape=(look_back, n_features))
#     pred = model(seq)

#     return Model(seq, pred)

In [132]:
def build_critic(look_back, n_features):
    model = Sequential()

    model.add(Conv1D(16, kernel_size=2, padding='same', input_shape=(look_back + 1, n_features))) # +1 => +target
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv1D(32, kernel_size=2, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv1D(64, kernel_size=2, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv1D(128, kernel_size=2, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    print('Critic summary:')
    model.summary()
    
    return model

#     seq = Input(shape=(look_back + 1, n_features))
#     valid = model(seq)

#     return Model(seq, valid)

In [133]:
def wasserstein_loss(y_true, y_pred):
    return K.mean(y_true * y_pred)

In [134]:
def build_GAN(look_back, n_features):
    
    critic = build_critic(look_back, n_features)
    critic.compile(loss=wasserstein_loss, 
                   optimizer=RMSprop(lr=0.1), 
                   metrics=['accuracy'])
    
    generator = build_generator(look_back, n_features)
    
    gen_input = Input(shape=(look_back, n_features))
    gen_output = generator(gen_input)
    gen_output = Reshape((1, 1))(gen_output)
    gen_output_plus_pred = concatenate([gen_input, gen_output], axis=1)
    
    critic.trainable = False
    
    valid = critic(gen_output_plus_pred)
    
    combined = Model(gen_input, valid)
    combined.compile(loss=wasserstein_loss, 
                     optimizer=RMSprop(lr=0.00001), 
                     metrics=['accuracy'])

    print('GAN summary:')
    combined.summary()
    
    return combined, generator, critic
    

In [135]:
GAN, generator, critic = build_GAN(look_back, n_features)

Critic summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_33 (Conv1D)           (None, 4, 16)             48        
_________________________________________________________________
leaky_re_lu_53 (LeakyReLU)   (None, 4, 16)             0         
_________________________________________________________________
dropout_51 (Dropout)         (None, 4, 16)             0         
_________________________________________________________________
conv1d_34 (Conv1D)           (None, 4, 32)             1056      
_________________________________________________________________
batch_normalization_34 (Batc (None, 4, 32)             128       
_________________________________________________________________
leaky_re_lu_54 (LeakyReLU)   (None, 4, 32)             0         
_________________________________________________________________
dropout_52 (Dropout)         (None, 4, 32)             0    

In [136]:
def get_batch(X, y, batch_idx, batch_size):
    X_batch = X[batch_idx:batch_idx+batch_size]
    y_batch = y[batch_idx:batch_idx+batch_size]
    
    return X_batch, y_batch

In [137]:
def get_real_and_fake_samples(X, y, generator):
    X_real = np.concatenate((X, y.reshape(-1, 1, 1)), axis=1)  
    y_pred = generator.predict(X)  
    X_fake = np.concatenate((X, y_pred.reshape(-1, 1, 1)), axis=1)
    
    return X_real, X_fake

In [138]:
def train_GAN(X, y, gan, generator, critic, n_epochs=100, batch_size=100, look_back=3, n_features=1, clip_value=0.01):
    data_len = len(X)
    total_batches = int(data_len / batch_size)
    n_points = look_back + 1 # look back + prediction

    for epoch in range(n_epochs):

        for batch_idx in range(total_batches):
            
            X_batch, y_batch = get_batch(X, y, batch_idx, batch_size)
            X_real, X_fake = get_real_and_fake_samples(X_batch, y_batch, generator)
            
            y_real = np.ones((batch_size, 1))
            y_fake = np.zeros((batch_size, 1))
            
            # Train critic
            d_loss_real = critic.train_on_batch(X_real, y_real)
            d_loss_fake = critic.train_on_batch(X_fake, y_fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            
            # Clip critic weights
            for l in critic.layers:
                weights = l.get_weights()
                weights = [np.clip(w, -clip_value, clip_value) for w in weights]
                l.set_weights(weights)

            # Train generator
            g_loss = gan.train_on_batch(X_batch, y_real)
            
        print ("Epoch %d/%d [D loss: %f] [G loss: %f]" % (epoch+1, n_epochs, 1 - d_loss[0], 1 - g_loss[0]))


In [139]:
train_GAN(X_train, y_train, GAN, generator, critic, n_epochs=100, batch_size=100)

Epoch 1/100 [D loss: 0.757293] [G loss: 0.514383]
Epoch 2/100 [D loss: 0.757323] [G loss: 0.514394]
Epoch 3/100 [D loss: 0.757299] [G loss: 0.514308]
Epoch 4/100 [D loss: 0.757285] [G loss: 0.514355]
Epoch 5/100 [D loss: 0.757304] [G loss: 0.514349]
Epoch 6/100 [D loss: 0.757300] [G loss: 0.514440]
Epoch 7/100 [D loss: 0.757288] [G loss: 0.514343]
Epoch 8/100 [D loss: 0.757300] [G loss: 0.514356]
Epoch 9/100 [D loss: 0.757295] [G loss: 0.514370]
Epoch 10/100 [D loss: 0.757303] [G loss: 0.514367]
Epoch 11/100 [D loss: 0.757290] [G loss: 0.514320]
Epoch 12/100 [D loss: 0.757340] [G loss: 0.514382]
Epoch 13/100 [D loss: 0.757327] [G loss: 0.514309]
Epoch 14/100 [D loss: 0.757300] [G loss: 0.514327]
Epoch 15/100 [D loss: 0.757308] [G loss: 0.514394]
Epoch 16/100 [D loss: 0.757311] [G loss: 0.514354]
Epoch 17/100 [D loss: 0.757291] [G loss: 0.514344]
Epoch 18/100 [D loss: 0.757324] [G loss: 0.514383]


KeyboardInterrupt: 